In [191]:
from elasticsearch import Elasticsearch

es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
es.indices.create("es0")

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'es0'}

In [192]:
es.index(index='es0', body={"content": "quick brother"})
es.index(index='es0', body={"content": "quick brother good"})
es.index(index='es0', body={"content": "quick brother happy"})
es.index(index='es0', body={"content": "brown fox good"})
es.index(index='es0', body={"content": "brown fox happy"})
es.index(index='es0', body={"content": "quiick"})
es.index(index='es0', body={"content": "brodcdmm"})
es.index(index='es0', body={"content": "Come on"})
es.index(index='es0', body={"content": "johnathon"})
es.index(index='es0', body={"content": "jonathon"})
es.index(index='es0', body={"content": "jonhbthon"})
es.index(index='es0', body={"content": "(1+1)=2"})

{'_index': 'es0',
 '_type': '_doc',
 '_id': '6MmCNoYBw-CyhYObAsCP',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 11,
 '_primary_term': 1}

In [193]:
body = {
    "query": {
        "query_string": {
            # 通配符
            # using ? to replace a single character, and * to replace zero or more characters
            # qu?ck 匹配quick;不匹配quiick
            # bro*  匹配brother, brodcdmm
            "query": "qu?ck bro*",
            "default_field": "content",
            "boost": 1
        }
    }
}
'''
default_field
    (Optional, string) Default field to search if no field is provided in the query string. Supports wildcards (*).

default_operator
    (Optional, string) Default boolean logic used to interpret text in the query string if no operators are specified. Valid values are:

    OR (Default)
        For example, a query string of capital of Hungary is interpreted as capital
        OR of OR Hungary.
    AND
        For example, a query string of capital of Hungary is interpreted as capital
        AND of AND Hungary.
'''
'''
GET es0/_search
`body`
'''
# minimum_should_match, fuzziness参数含义参考`match`
es.search(index='es0', body=body)

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 6, 'relation': 'eq'},
  'max_score': 2.0,
  'hits': [{'_index': 'es0',
    '_type': '_doc',
    '_id': '3cmCNoYBw-CyhYObAcD4',
    '_score': 2.0,
    '_source': {'content': 'quick brother'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '3smCNoYBw-CyhYObAsBD',
    '_score': 2.0,
    '_source': {'content': 'quick brother good'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '38mCNoYBw-CyhYObAsBO',
    '_score': 2.0,
    '_source': {'content': 'quick brother happy'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '4MmCNoYBw-CyhYObAsBV',
    '_score': 1.0,
    '_source': {'content': 'brown fox good'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '4cmCNoYBw-CyhYObAsBd',
    '_score': 1.0,
    '_source': {'content': 'brown fox happy'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '48mCNoYBw-CyhYObAsBs',
    '_score': 1.0

In [194]:
es.search(index='es0', body={
    "query": {
        "query_string": {
            # 正则表达式:https://www.elastic.co/guide/en/elasticsearch/reference/7.11/regexp-syntax.html
            # Regular expression patterns can be embedded in the query string by wrapping them in forward-slashes ("/"):
            # /joh?n(ath[oa]n)/ 匹配johnathon, jonathon; 不匹配jonhbthon(===> [oa]部分不匹配)
            "query": "/joh?n(ath[oa]n)/",
            "default_field": "content"
        }
    }
})

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'es0',
    '_type': '_doc',
    '_id': '5cmCNoYBw-CyhYObAsB7',
    '_score': 1.0,
    '_source': {'content': 'johnathon'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '5smCNoYBw-CyhYObAsCB',
    '_score': 1.0,
    '_source': {'content': 'jonathon'}}]}}

In [195]:
es.search(index='es0', body={
    "query": {
        "query_string": {
            # 布尔运算符(多个运算符时使用括号进行分组): AND OR NOT
            # 匹配quick brother, 匹配qucik brother happy, brown fox happy
            # 不匹配brodcdmm(===> qu?ck AND bro*), quick brother good(===> AND NOT good), brown fox good(===> AND NOT good)
            "query": "((qu?ck AND bro*) OR (brown AND fox)) AND NOT good",
            "default_field": "content"
        }
    }
})

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 3, 'relation': 'eq'},
  'max_score': 2.7373953,
  'hits': [{'_index': 'es0',
    '_type': '_doc',
    '_id': '4cmCNoYBw-CyhYObAsBd',
    '_score': 2.7373953,
    '_source': {'content': 'brown fox happy'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '3cmCNoYBw-CyhYObAcD4',
    '_score': 2.0,
    '_source': {'content': 'quick brother'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '38mCNoYBw-CyhYObAsBO',
    '_score': 2.0,
    '_source': {'content': 'quick brother happy'}}]}}

In [196]:
es.search(index='es0', body={
    "query": {
        "query_string": {
            # 转义字符(使用反斜杠`\`进行转义)
            # The reserved characters are: + - = && || > < ! ( ) { } [ ] ^ " ~ * ? : \ /
            "query": "\(1\+1\)\=2",
            "default_field": "content"
        }
    }
})

{'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 6.9992065,
  'hits': [{'_index': 'es0',
    '_type': '_doc',
    '_id': '6MmCNoYBw-CyhYObAsCP',
    '_score': 6.9992065,
    '_source': {'content': '(1+1)=2'}}]}}

In [197]:
es.indices.delete('es0')

{'acknowledged': True}